In [3]:
%pip install --upgrade torch




Note: you may need to restart the kernel to use updated packages.


In [5]:
# Import dependencies
import cv2
import torch
import matplotlib.pyplot as plt 
# from midas.dpt_depth import DPTDepthModel
from MiDaS.midas.dpt_depth import DPTDepthModel
# from midas.midas_net import MidasNet
from MiDaS.midas.midas_net import MidasNet
# from midas.midas_net_custom import MidasNet_small
from MiDaS.midas.midas_net_custom import MidasNet_small
# from midas.transforms import Resize, NormalizeImage, PrepareForNet
from MiDaS.midas.transforms import Resize, NormalizeImage, PrepareForNet

# Download the MiDaS
midas = torch.hub.load('intel-isl/MiDaS', 'MiDaS_small')
midas.to('cpu')
midas.eval()
# Input transformation pipeline
transforms = torch.hub.load('intel-isl/MiDaS', 'transforms')
transform = transforms.small_transform 

# Hook into OpenCV
cap = cv2.VideoCapture(0)
while cap.isOpened(): 
    ret, frame = cap.read()

    # Transform input for midas 
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    imgbatch = transform(img).to('cpu')

    # Make a prediction
    with torch.no_grad(): 
        prediction = midas(imgbatch)
        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size = img.shape[:2], 
            mode='bicubic', 
            align_corners=False
        ).squeeze()

        output = prediction.cpu().numpy()

        print(output)
    plt.imshow(output)
    cv2.imshow('CV2Frame', frame)
    plt.pause(0.00001)

    if cv2.waitKey(10) & 0xFF == ord('q'): 
        cap.release()
        cv2.destroyAllWindows()

plt.show()

ModuleNotFoundError: No module named 'MiDaS'

In [10]:
import pandas as pd
import os

# list to store dataframes
df_list = []

# loop through each file in the directory
for file in os.listdir("data/Payment History"):
    # skip files that are not csv
    if file.endswith(".csv"):
        # read csv file into a dataframe
        df = pd.read_csv(os.path.join("data/Payment History", file))
        # add the dataframe to the list
        df_list.append(df)

# combine all the dataframes into a single dataframe
df = pd.concat(df_list, ignore_index=True)

# drop duplicates from the dataframe
df.drop_duplicates(inplace=True)
df["Fecha de pago"] = pd.to_datetime(df["Fecha de pago"], format="%d/%m/%Y")
df["Fecha de Inicio"] = pd.to_datetime(df["Fecha de Inicio"], format="%d/%m/%Y")
df.sort_values("Fecha de pago", inplace=True)
# save the dataframe to a csv file
df.to_csv("combined_csv.csv", index=False)


In [11]:
df = pd.read_csv('combined_csv.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3216 entries, 0 to 3215
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   N°               3216 non-null   int64 
 1   Cliente:         3216 non-null   object
 2   Email:           3216 non-null   object
 3   Estado:          3216 non-null   object
 4   Plan             3216 non-null   object
 5   Fecha de pago    3216 non-null   object
 6   Fecha de Inicio  3216 non-null   object
 7   Tipo             3216 non-null   object
 8   Monto            3216 non-null   object
 9   Vendedor/a       3216 non-null   object
dtypes: int64(1), object(9)
memory usage: 251.4+ KB


In [17]:
import os
import stat
# loop through each file in the directory
for file in os.listdir("data/Payment History"):
    # skip files that are not csv
    if file.endswith(".csv"):
        permissions = os.stat(file).st_mode
        print(stat.filemode(permissions))





FileNotFoundError: [Errno 2] No such file or directory: 'BoxMagic - 2023-01-04T205100.611.csv'

In [1]:
import os
folder =  "data/Payment History"
# Change to the directory containing the files
os.chdir(folder)

# Iterate over the files in the directory
for index, file in enumerate(os.listdir()):
    # Split the file name and extension
    name, extension = os.path.splitext(file)

    # Generate the new file name
    new_name = f"{index}{extension}"

    # Rename the file
    os.rename(file, new_name)
